In [1]:
import time
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, timedelta
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [2]:
s = 2020
players = pd.read_csv('../data/database-players.csv')
players = players[players['Last_Year'] >= s]
nfl_weeks = pd.read_csv('../data/NFL-Week-Dates.csv')
nfl_weeks['Week'] = nfl_weeks['Week'].astype(str)
def pre_thu(d):
    days_behind = 3 - d.weekday()
    if days_behind > 0:
        days_behind -= 7
    return d + dt.timedelta(days_behind)
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)
def renaming_cols(col):
    if "Unnamed:" in col:
        new = col[findnth(col, "'", 2)+1:-2]
    elif "Passing" in col or "Rushing" in col or "Receiving" in col or "Fumbles" in col or "Scoring" in col or "Tackles" in col or "Punting" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    elif "Punt Returns" in col or "ST Snaps" in col or "Kick Returns" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    elif "Def. Snaps" in col or "Off. Snaps" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)-1] + "_" + col[findnth(col, " ", 0)+1:findnth(col, "'", 1)] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    elif "Def Interceptions" in col:
        new = col[findnth(col, "'", 0)+1:findnth(col, " ", 0)] + "_" + col[findnth(col, " ", 0)+1:findnth(col, " ", 0)+4] + "_" + col[findnth(col, "'", 2)+1:findnth(col, "'", 3)]
    else:
        new = col
    return new
def player_address_rename(col):
    if "Player_Address" in col:
        new = col[:len("Player_Address")]
    else:
        new = col
    return new

In [ ]:
# player stats clean
start = time.time()
stats = pd.read_csv(f'../data/raw-data/player-weekly-stats-{s}-raw.csv', low_memory=False)
cols = []
for c in stats.columns:
    cols.append(c)
cols.remove('Player_Address')
cols.remove("('Unnamed: 3_level_0', 'Week')")
# stats = stats.groupby(['Player_Address', "('Unnamed: 3_level_0', 'Week')"])[cols].apply(lambda x: x.fillna(method="ffill").fillna(method="bfill").drop_duplicates()).reset_index
stats = stats.rename(columns=renaming_cols)
stats = stats.rename(columns = {"Unnamed: 6_level_1": "Home_Away", "Tm": "Team", "Passing_Yds.1": "Passing_Sk_Yds", "GS": "Gm_Start", "G#": "Gm_Num"})
stats = pd.merge(left=stats, 
                 right=players, 
                 how='outer', 
                 left_on='Player_Address', 
                 right_on='Address')
stats = stats.groupby(stats.columns, axis=1).sum()
stats.drop(['First_Year', 'Last_Year', 'Rk', 'Address'], axis=1, inplace=True)
stats.replace({'Home_Away': {'@': 'Away', 0: 'Home'}, 
               'Gm_Start': {'*': True, 0: False}}, inplace=True)
stats.dropna(subset=['Player_Address'], inplace=True)
stats[['Off_Snaps_Pct', 
       'Def_Snaps_Pct', 
       'ST_Snaps_Pct',
       'Player_Address', 
       'Week']] = stats[['Off_Snaps_Pct', 
                         'Def_Snaps_Pct', 
                         'ST_Snaps_Pct', 
                         'Player_Address', 
                         'Week']].astype(str)
stats['Off_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Off_Snaps_Pct'].values))
stats['Def_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['Def_Snaps_Pct'].values))
stats['ST_Snaps_Pct'] = list(map(lambda x: x[:-1], stats['ST_Snaps_Pct'].values))
stats['Player_Address'] = list(map(lambda x: x[11:], stats['Player_Address'].values))
stats['Date'] = pd.to_datetime(stats['Date'], errors='coerce', format='%Y-%m-%d')
stats = stats[['Player_Address', 'Player', 'Position', 'Age', 
               'Team', 'Home_Away', 'Opp', 'Result', 'Week', 
               'Gm_Num', 'Season', 'Gm_Start', 'Date', 'Off_Snaps_Num',
               'Off_Snaps_Pct', 'Def_Snaps_Num', 'Def_Snaps_Pct',
               'ST_Snaps_Num', 'ST_Snaps_Pct', 'Passing_Att',  'Passing_Cmp',
               'Passing_Cmp%', 'Passing_Yds', 'Passing_TD', 'Passing_Int',
               'Passing_Rate', 'Passing_Sk', 'Passing_Sk_Yds', 'Passing_Y/A',
               'Passing_AY/A', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Y/A',
               'Rushing_TD', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD',
               'Receiving_Y/R', 'Receiving_Ctch%', 'Receiving_Y/Tgt', 'Kick_Returns_Rt',
               'Kick_Returns_Yds', 'Kick_Returns_Y/Rt', 'Kick_Returns_TD', 'Punt_Returns_Ret', 'Punt_Returns_Yds',
               'Punt_Returns_TD', 'Punt_Returns_Y/R', 'Punting_Pnt', 'Punting_Yds', 'Punting_Y/P',
               'Punting_Blck', 'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM',
               'Scoring_XPA', 'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA',
               'Scoring_FG%', 'Scoring_2PM', 'Scoring_Sfty', 'Sk',
               'Tackles_Solo', 'Tackles_Ast', 'Tackles_Comb', 'Tackles_TFL',
               'Tackles_QBHits', 'Def_Int_Int', 'Def_Int_Yds', 'Def_Int_TD',
               'Def_Int_PD', 'Fumbles_Fmb', 'Fumbles_FL', 'Fumbles_FF',
               'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD']]
for c in stats.columns[13:]:
    stats[c] = pd.to_numeric(stats[c], errors='coerce')
stats.to_csv(f'../data/clean-data/stats-{s}.csv', index=False)

# player injurys clean
injury = pd.read_csv(f'../data/raw-data/nfl-injury-report-{s}-raw.csv', low_memory=False)
cols = [col for col in injury.columns if "Player_Address" in col]
injury["Player_Address"] = injury[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
injury = injury.drop(cols[1:], axis=1)
injury = pd.melt(injury,
                 id_vars=['Player', 'Team', 'Season', 'Player_Address'], 
                 var_name='Date', 
                 value_name='Status')
injury[['Date','Opp']] = injury.Date.str.split('vs. ', expand=True)
injury[['Month','Day']] = injury.Date.str.split('/', expand=True)
injury[['Status','Injury']] = injury.Status.str.split(":", expand=True)
injury.dropna(axis=0, subset=['Status','Injury'], how='all', inplace=True)
injury[['Season', 'Month', 'Day']] = injury[['Season', 'Month', 'Day']].astype(int)
injury['Date'] = injury['Date'] + '/' + (np.where(injury['Month'] <= 2, injury['Season'] + 1, injury['Season'])).astype(str)
injury['Date'] = pd.to_datetime(injury['Date'])
injury['week_start_nfl'] = injury['Date'].apply(pre_thu)
nfl_weeks['Start Date'] = pd.to_datetime(nfl_weeks['Start Date'])
injury = pd.merge(left=injury, 
                  right=nfl_weeks, 
                  how='left', 
                  left_on='week_start_nfl', 
                  right_on='Start Date')
injury.replace({'Team':
                   {'crd': 'ARI', 'atl': 'ATL', 'rav': 'BAL', 'buf': 'BUF', 
                    'car': 'CAR', 'chi': 'CHI', 'cin': 'CIN', 'cle': 'CLE', 
                    'dal': 'DAL', 'den': 'DEN', 'det': 'DET', 'gnb': 'GNB',
                    'htx': 'HOU', 'clt': 'IND', 'jax': 'JAX', 'kan': 'KAN', 
                    'sdg': 'LAC', 'ram': 'LAR', 'mia': 'MIA', 'min': 'MIN', 
                    'nor': 'NOR', 'nwe': 'NWE', 'nyg': 'NYG', 'nyj': 'NYJ', 
                    'rai': 'OAK', 'phi': 'PHI', 'pit': 'PIT', 'sea': 'SEA',
                    'sfo': 'SFO', 'tam': 'TAM', 'oti': 'TEN', 'was': 'WAS'}},
               inplace=True)
injury['Injury'] = injury['Injury'].str.strip(' ')
injury.replace({"Player_Address": {"_": "", "nan": ""}}, regex=True, inplace=True)
injury = pd.merge(left=injury,  
                  right=players, 
                  how='outer', 
                  left_on='Player_Address',
                  right_on='Address')
injury.rename(columns={"Player_x": "Player"}, inplace=True)
injury.drop(['Month', 'Day', 'week_start_nfl', 'Start Date', 'First_Year', 'Last_Year', 'Player_y'], axis=1, inplace=True)
injury['Date'] = pd.to_datetime(injury['Date'], errors='coerce', format='%Y-%m-%d')
injury.dropna(subset=['Player_Address'], inplace=True)
injury.dropna(subset=['Week'], inplace=True)
injury['Player_Address'] = list(map(lambda x: x[11:], injury['Player_Address'].values))
injury = injury[["Player", "Team", "Season", "Player_Address", "Date", "Status", "Opp", "Injury", "Week", "Position"]]
injury.to_csv(f'../data/clean-data/injury-{s}.csv', index=False)

# combine stats and injuries
stats = pd.read_csv(f'../data/clean-data/stats-{s}.csv')
injury = pd.read_csv(f'../data/clean-data/injury-{s}.csv')
nfl_data = stats.merge(injury, 
                       how="outer",
                       on=["Player_Address", "Date"])
nfl_data["Player_x"] = nfl_data["Player_x"].fillna(nfl_data["Player_y"])
nfl_data["Position_x"] = nfl_data["Position_x"].fillna(nfl_data["Position_y"])
nfl_data["Team_x"] = nfl_data["Team_x"].fillna(nfl_data["Team_y"])
nfl_data["Opp_x"] = nfl_data["Opp_x"].fillna(nfl_data["Opp_y"])
nfl_data["Week_x"] = nfl_data["Week_x"].fillna(nfl_data["Week_y"])
nfl_data["Season_x"] = nfl_data["Season_x"].fillna(nfl_data["Season_y"])
nfl_data = nfl_data.rename(columns={"Player_x": "Player", 
                                    "Position_x": "Position", 
                                    "Team_x": "Team", 
                                    "Opp_x": "Opp", 
                                    "Date_x": "Date", 
                                    "Season_x": "Season", 
                                    "Week_x": "Week"})
nfl_data = nfl_data[['Player_Address', 'Player', 'Position', 'Age', 
                     'Team', 'Home_Away', 'Opp', 'Result', 'Date', 'Week', 
                     'Gm_Num', 'Season', 'Status', 'Injury', 'Gm_Start', 'Off_Snaps_Num',
                     'Off_Snaps_Pct', 'Def_Snaps_Num', 'Def_Snaps_Pct',
                     'ST_Snaps_Num', 'ST_Snaps_Pct', 'Passing_Att',  'Passing_Cmp',
                     'Passing_Cmp%', 'Passing_Yds', 'Passing_TD', 'Passing_Int',
                     'Passing_Rate', 'Passing_Sk', 'Passing_Sk_Yds', 'Passing_Y/A',
                     'Passing_AY/A', 'Rushing_Att', 'Rushing_Yds', 'Rushing_Y/A',
                     'Rushing_TD', 'Receiving_Tgt', 'Receiving_Rec', 'Receiving_Yds', 'Receiving_TD',
                     'Receiving_Y/R', 'Receiving_Ctch%', 'Receiving_Y/Tgt', 'Kick_Returns_Rt',
                     'Kick_Returns_Yds', 'Kick_Returns_Y/Rt', 'Kick_Returns_TD', 'Punt_Returns_Ret', 'Punt_Returns_Yds',
                     'Punt_Returns_TD', 'Punt_Returns_Y/R', 'Punting_Pnt', 'Punting_Yds', 'Punting_Y/P',
                     'Punting_Blck', 'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM',
                     'Scoring_XPA', 'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA',
                     'Scoring_FG%', 'Scoring_2PM', 'Scoring_Sfty', 'Sk',
                     'Tackles_Solo', 'Tackles_Ast', 'Tackles_Comb', 'Tackles_TFL',
                     'Tackles_QBHits', 'Def_Int_Int', 'Def_Int_Yds', 'Def_Int_TD',
                     'Def_Int_PD', 'Fumbles_Fmb', 'Fumbles_FL', 'Fumbles_FF',
                     'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD']]
nfl_data.to_csv(f'../data/clean-data/nfl-{s}.csv', index=False)

# create a database of all athletes that sustained an injury and what their production was post injury for the remainder of the season

nfl_data = pd.read_csv(f'../data/clean-data/nfl-{s}.csv', low_memory=False)
nfl_data.sort_values(['Player_Address','Week'], inplace=True)
nfl_data.reset_index(inplace=True)
nfl_data.drop(['index'], axis=1, inplace=True)

status = nfl_data['Status'].dropna().unique().tolist()
player_list = nfl_data.loc[nfl_data['Status'].isin(status)].drop_duplicates(subset=['Player_Address']).index.unique().tolist()

injury_weeks = pd.DataFrame()
healthy_weeks = pd.DataFrame()

for i in range(0,len(player_list)):
    idx = nfl_data[nfl_data['Player_Address'] == nfl_data['Player_Address'][player_list[i]]].tail(1)
    inj = nfl_data.loc[player_list[i]-1: idx.index.values.astype(int)[0]]
    injury_weeks = pd.concat([injury_weeks, inj])
injury_weeks['Inj_Week'] = 0
injury_weeks['Inj_Week'] = injury_weeks.groupby('Player_Address').cumcount()

for i in range(0,len(player_list)):
    idx = nfl_data[nfl_data['Player_Address'] == nfl_data['Player_Address'][player_list[i]]].head(1)
    healthy = nfl_data.loc[idx.index.values.astype(int)[0]:player_list[i]-1]
    healthy_weeks = pd.concat([healthy_weeks, healthy])
healthy_weeks['Healthy_Week'] = 0
healthy_weeks['Healthy_Week'] = healthy_weeks.groupby('Player_Address').cumcount()+1

injury_weeks.to_csv(f'../data/clean-data/weeks-injured-{s}.csv', index=False)
healthy_weeks.to_csv(f'../data/clean-data/weeks-healthy-{s}.csv', index=False)


end = time.time()
print(f'Done with season {s}. Time to complete: {end-start} seconds.')

In [ ]:
# s = 2020
# i = pd.read_csv(f'../data/clean-data/weeks-injured-{s}.csv')
# h = pd.read_csv(f'../data/clean-data/weeks-healthy-{s}.csv')
# c = i.merge(h, how="outer", on=['Player_Address', 'Player', 'Position', 'Age', 'Team', 'Home_Away',
#        'Opp', 'Result', 'Date', 'Week', 'Gm_Num', 'Season', 
#        'Gm_Start', 'Off_Snaps_Num', 'Off_Snaps_Pct', 'Def_Snaps_Num',
#        'Def_Snaps_Pct', 'ST_Snaps_Num', 'ST_Snaps_Pct', 'Passing_Att',
#        'Passing_Cmp', 'Passing_Cmp%', 'Passing_Yds', 'Passing_TD',
#        'Passing_Int', 'Passing_Rate', 'Passing_Sk', 'Passing_Sk_Yds',
#        'Passing_Y/A', 'Passing_AY/A', 'Rushing_Att', 'Rushing_Yds',
#        'Rushing_Y/A', 'Rushing_TD', 'Receiving_Tgt', 'Receiving_Rec',
#        'Receiving_Yds', 'Receiving_TD', 'Receiving_Y/R', 'Receiving_Ctch%',
#        'Receiving_Y/Tgt', 'Kick_Returns_Rt', 'Kick_Returns_Yds',
#        'Kick_Returns_Y/Rt', 'Kick_Returns_TD', 'Punt_Returns_Ret',
#        'Punt_Returns_Yds', 'Punt_Returns_TD', 'Punt_Returns_Y/R',
#        'Punting_Pnt', 'Punting_Yds', 'Punting_Y/P', 'Punting_Blck',
#        'Scoring_TD', 'Scoring_Pts', 'Scoring_XPM', 'Scoring_XPA',
#        'Scoring_XP%', 'Scoring_FGM', 'Scoring_FGA', 'Scoring_FG%',
#        'Scoring_2PM', 'Scoring_Sfty', 'Sk', 'Tackles_Solo', 'Tackles_Ast',
#        'Tackles_Comb', 'Tackles_TFL', 'Tackles_QBHits', 'Def_Int_Int',
#        'Def_Int_Yds', 'Def_Int_TD', 'Def_Int_PD', 'Fumbles_Fmb', 'Fumbles_FL',
#        'Fumbles_FF', 'Fumbles_FR', 'Fumbles_Yds', 'Fumbles_TD'])
# c['Status_x'] = c['Status_x'].fillna(c['Status_y'])
# c['Injury_x'] = c['Injury_x'].fillna(c['Injury_y'])
# c.rename(columns={'Status_x': 'Status',
#                   'Injury_x': 'Injury'},
#         inplace=True)
# c.drop(['Injury_y','Status_y'], axis=1, inplace=True)

In [12]:
test = pd.read_csv('test.csv', index_col=0, low_memory=False)

In [ ]:
cols = []
for c in test.columns:
    cols.append(c)
cols.remove('Player_Address')
cols.remove('Week')
t = test.groupby(['Player_Address', 'Week'])[cols].apply(lambda x: x.ffill().bfill())

In [ ]:
test.loc[:,cols] = t.loc[:,cols]

In [ ]:
# test.drop_duplicates(inplace=True)

In [ ]:
test = test.reset_index(drop=True)

In [ ]:
dups = test[test.duplicated()]

In [ ]:
test.to_csv('test1.csv')

In [ ]:
dups.to_csv('dups.csv')